In [ ]:
!pip install pandas matplotlib

In [ ]:
import pickle, os
import pandas as pd

In [ ]:
dataset = None
params = []
results = ["num_total_events","user","sys"]

with open("results.pickle", "rb") as f:
    dataset = pickle.load(f)
with open("simulation.plan", "rt") as f:
    params_dict = {}
    exec(f.read(), params_dict)
    params = [k for k in params_dict.keys() if not k.startswith("_") and k != "title"]
    
print("params: ",params)
print(len(dataset), "results found")
df = pd.DataFrame([dataset[k] for k in dataset])
removed_keys = [k for k in df.columns if len(df[k].unique())  == 1]
params = [k for k in params if k not in removed_keys]
print(df.dtypes)

In [ ]:
keys = [k for k in df.columns if k not in removed_keys]
df[keys]